In [14]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from astropy.io import ascii 
from astropy.table import Table

In [15]:
def import_snana(filename):
    start_data = False
    table = None
    with open(filename) as f:
        for i,line in enumerate(f):
            if 'VARLIST' in line:
                header = line.split()
                header = header[0:9]
                dummy = [['X'*20]]*len(header)
                table = Table(dummy, names=header).copy()[:0]
            elif table is not None:
                data=line.split()
                data=data[0:9]
                if len(data)!=9:
                    break
                table.add_row(data)
    mask = np.array([r['MAGERR'] is not None for r in table])
    table = table[mask]
    table['MJD']    = table['MJD'].astype(float)
    table['FLT'] = table['FLT'].astype(str)
    table['FLUXCAL'] = table['MAGERR'].astype(float)
    table['FLUXCALERR'] = table['MAGERR'].astype(float)
    table['MAG'] = table['MAG'].astype(float)
    table['MAGERR'] = table['MAGERR'].astype(float)
    return(table)

In [16]:
hgw = import_snana('2020hgw_data.snana.txt')
jfo = import_snana('2020jfo_lightcurves')
jww = import_snana('2020jww_data.snana.txt')
rth = import_snana('2020rth_data.snana.txt')
tly = import_snana('2020tly_YSEdata.snana.txt')

In [17]:
z = {'hgw':0.043, 'jfo':0.005, 'jww':0.046, 'rth':0.0184, 'tly':0.058}
ebv = {'hgw':0.011, 'jfo':0.019, 'jww':0.035, 'rth':0.041, 'tly':0.071}

In [18]:
all_names = ['2020hgw', '2020jfo', '2020jww', '2020rth', '2020tly']
all_data = [hgw, jfo, jww, rth, tly]
all_data_df = [] 
for object in all_data:
    df = object.to_pandas()
    # clean up data - remove missing values 
    # also want to drop large errors 
    clean = df.dropna()
    cleaner = clean[clean['MAGERR'] <= 1]
    all_data_df.append(cleaner)
    

In [28]:
def superbol_inputs(names, dfs):
    for idx, df in enumerate(dfs):
        df = df.replace({'g-ZTF':'g', 'r-ZTF':'r', 'orange-ATLAS':'o', 'cyan-ATLAS':'c', 'G':'E', 'UVW1':'A', 'UVM2':'D', 'UVW2':'S', 'i-N':'i', 'gp':'g', 'ip':'i','rp':'r', 'up':'u', 'U_S':'u'})
        df = df[['MJD', 'MAG', 'MAGERR', 'FLT']]
        filters = df['FLT'].unique()
        print(names[idx], filters)
        for filter in filters:
            filter_df = df[df['FLT'] == filter]
            filename = names[idx] + '_' + str(filter) + '.txt'
            filter_df[['MJD', 'MAG', 'MAGERR']].to_csv(filename, sep = ' ', header = False, index = False)
            print(names[idx], filter_df, len(filter_df))

In [29]:
superbol_inputs(all_names, all_data_df)

2020hgw ['g' 'r' 'o' 'i' 'z' 'A' 'U' 'B' 'S' 'V' 'D']
2020hgw            MJD     MAG  MAGERR FLT
0    58954.387  17.882   0.067   g
2    58955.404  17.780   0.055   g
5    58956.369  17.617   0.032   g
8    58964.345  17.559   0.050   g
9    58967.288  17.679   0.069   g
11   58970.307  17.808   0.066   g
13   58973.267  17.903   0.067   g
15   58975.275  18.056   0.137   g
16   58976.201  18.091   0.092   g
18   58979.240  18.188   0.066   g
20   58985.299  18.369   0.083   g
22   58991.307  18.462   0.073   g
23   58994.244  18.516   0.064   g
24   58997.221  18.653   0.087   g
26   59000.246  18.675   0.085   g
27   59000.266  18.700   0.103   g
28   59004.203  18.825   0.130   g
31   59009.243  19.001   0.131   g
32   59012.206  19.061   0.101   g
34   59016.208  19.129   0.093   g
36   59019.260  19.163   0.135   g
39   59025.237  19.321   0.069   g
41   59029.207  19.313   0.182   g
45   59037.224  19.564   0.216   g
52   59045.236  19.816   0.178   g
56   59048.267  19.835   0.1

In [21]:
mask = (rth['FLT'] == 'g-ZTF') | (rth['FLT'] == 'g')
masked_rth = rth[mask].to_pandas()

In [22]:
no_na = masked_rth.dropna()
all_g = no_na.replace({'g-ZTF':'g'})

In [23]:
def superbol_inputs(names, dfs):
    for idx, df in enumerate(dfs):
        df = df.replace({'g-ZTF':'g', 'r-ZTF':'r', 'orange-ATLAS':'o', 'cyan-ATLAS':'c'})
        df = df[['MJD', 'MAG', 'MAGERR', 'FLT']]
        filters = df['FLT'].unique()
        print(names[idx], filters)
        for filter in filters:
            if (filter == 'o') | (filter == 'c'):
                continue
            filter_df = df[df['FLT'] == filter]
            filename = names[idx] + '_' + str(filter) + '.txt'
            filter_df[['MJD', 'MAG', 'MAGERR']].to_csv(filename, sep = ' ', header = False, index = False)
            print(names[idx], filter_df)

In [24]:
def single_input(name, df):
    df = df.replace({'g-ZTF':'g', 'r-ZTF':'r', 'orange-ATLAS':'o', 'cyan-ATLAS':'c'})
    df = df[['MJD', 'MAG', 'MAGERR', 'FLT']]
    filters = df['FLT'].unique()
    for filter in filters:
        if (filter == 'o') | (filter == 'c'):
            continue
        filter_df = df[df['FLT'] == filter]
        filename = str(name) + '_' + str(filter) + '.txt'
        filter_df[['MJD', 'MAG', 'MAGERR', 'FLT']].to_csv(filename, sep = ' ', header = False, index = False)
        print(filter_df, 'len_filter_df:', len(filter_df))
        

In [25]:
single_input('2020rth', all_data_df[3])

           MJD     MAG  MAGERR FLT
0    59084.428  17.179   0.054   g
1    59088.494  17.208   0.043   g
3    59091.417  17.267   0.038   g
4    59092.428  17.340   0.054   g
7    59094.500  17.380   0.053   g
10   59106.519  17.927   0.096   g
11   59108.370  17.993   0.086   g
13   59111.396  18.112   0.060   g
15   59114.352  18.268   0.075   g
19   59118.479  18.388   0.062   g
20   59120.363  18.453   0.092   g
21   59120.440  18.409   0.058   g
22   59121.365  18.383   0.090   g
23   59122.362  18.554   0.131   g
25   59123.400  18.431   0.108   g
27   59125.397  18.406   0.110   g
30   59130.405  18.544   0.082   g
33   59136.406  18.653   0.070   g
36   59140.413  18.712   0.096   g
38   59142.407  18.756   0.069   g
39   59144.396  18.811   0.068   g
42   59151.319  18.700   0.139   g
46   59155.361  18.852   0.114   g
47   59157.371  18.713   0.100   g
49   59158.320  18.949   0.147   g
51   59165.300  18.974   0.097   g
55   59169.332  19.090   0.094   g
56   59171.260  19.2